In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import redditutils as ru
import pickle


from string import punctuation

from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from scipy.io import mmwrite, mmread

from gensim import corpora, models, similarities, matutils

from xgboost import XGBClassifier

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train = pd.read_csv('train.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
test = pd.read_csv('test.csv', index_col = 0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
test.head()

,author,title,subreddit,score,num_comments,retrieved_on,id,created_utc,selftext,quality
700000,livesNbox,I wonder if anyone sells dandelion seeds,Showerthoughts,3.0,2.0,1.494614e+09,66dags,2017-04-19 21:33:46,NaN,1.0
700001,keepmovingon69,"Before phone and telegram lines, you could pro...",Showerthoughts,3.0,2.0,1.463498e+09,4c0ksq,2016-03-26 09:01:46,NaN,1.0
700002,vorpike,As an Asian idk why we stereotypically pronoun...,Showerthoughts,3.0,2.0,1.473036e+09,4x6gd8,2016-08-11 05:17:10,Some of us can't roll our tongues and pronounc...,1.0
700003,CappyCrunch,When you're at someone else's house the loudes...,Showerthoughts,3.0,2.0,1.492418e+09,5zt2xp,2017-03-16 20:28:14,NaN,1.0
700004,Fart_Kontrol,Why didn't Uncle Phillip buy the Fresh Prince'...,Showerthoughts,3.0,2.0,1.492411e+09,5zevhw,2017-03-14 20:56:38,"I mean, Fresh P is in high school. Seems like...",1.0


In [6]:
train.dropna(inplace=True)

In [7]:
train['quality'] = train['quality'].astype(int)

In [8]:
resp = train['quality']
feat = train['title']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(feat, resp, stratify = resp)

In [10]:
X_train_short = X_train[:50000]
y_train_short = y_train[:50000]

In [11]:
# English words
words_corpus = set(words.words())
# Stop words
stop = set(stopwords.words('english'))
# English words minus stop words
acceptable_words = words_corpus - stop
analyzer = CountVectorizer().build_analyzer()
stem = SnowballStemmer('english')

def english_corpus(doc, stemmer=stem):
    return [stemmer.stem(w) for w in analyzer(doc) if w in acceptable_words]

tfidf = TfidfVectorizer(stop_words = "english",
                        strip_accents = 'ascii',
                        max_df = .10,
                        min_df = 3, 
                        tokenizer = english_corpus,
                        ngram_range=(1, 2))

In [12]:
X_train_short[0]

"Every time I explain something to my five-year-old, I have to explain it like he's five."

In [14]:
X_tf = cv.fit_transform(X_train_short)

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  sorted(inconsistent))


In [38]:
from collections import Counter

In [41]:
Counter(km.labels_)

Counter({3: 4093,
         5: 34425,
         9: 3408,
         6: 1374,
         4: 1561,
         2: 1826,
         8: 1438,
         1: 1791,
         0: 83,
         7: 1})

In [40]:
km = KMeans(n_clusters = 10)

km.fit

array([[4.10705556, 4.08615265, 4.07685019, ..., 7.48331477, 4.08688647,
        4.12808344],
       [2.66172366, 2.66872706, 2.67983135, ..., 6.92820323, 2.71916878,
        2.68789106],
       [3.59724819, 3.58961904, 3.58326185, ..., 7.34846923, 3.63057617,
        3.30785346],
       ...,
       [3.62394338, 3.61071119, 3.60990877, ..., 7.34846923, 3.64948963,
        3.63456834],
       [2.85605707, 2.86107597, 2.87655971, ..., 7.        , 2.90991277,
        2.89274106],
       [2.03888744, 2.04509306, 1.46058321, ..., 6.55743852, 2.10938656,
        2.09627272]])

In [42]:
features = km.labels_

features = pd.get_dummies(features)

In [34]:
features_res, response_res = ru.upsample(features, np.array(y_train_short))

In [44]:
rfc = RandomForestClassifier(n_estimators = 100)

cross_val_score(rfc, features, y_train_short, cv=3, scoring='roc_auc')

array([0.51837419, 0.51947646, 0.52538815])